In [1]:
import numpy as np
import pandas as pd

In [2]:
lang_list = ['es']
lang = 'es'

In [3]:
test_df = pd.read_csv('../data/submissions/test9537.csv')
language_df = test_df[test_df.lang.isin(lang_list)]
language_df.columns = ['id', 'comment_text', 'lang', 'toxic']
language_df.to_csv('../data/{}_test.csv'.format(lang), index=False)
# language_df = language_df[(language_df['toxic'] > 0.8) | (language_df['toxic'] < 0.2)] #!!!!!!

language_df

,id,comment_text,lang,toxic
7,7,el skate es unos de los deportes favoritos de ...,es,0.046024
8,8,Me doy la bienvenida. A este usuari le gusta c...,es,0.269433
9,9,"ES NOTABLEMENTE TENDENCIOSO, NO SE HABLA DE CU...",es,0.281472
22,22,"El Jardín de infantes Nº938, fundado en 1989, ...",es,0.003624
49,49,Daré explicaciones y/o aclaraciones a cualquie...,es,0.154248
...,...,...,...,...
63783,63783,Los mongoles emplearon una crueldad apabullant...,es,0.046097
63787,63787,I N S T I - TI! T U - TU! T O - TO! INSTITUTO ...,es,0.380755
63804,63804,Te agradezco tu cordial mensaje. De los cambio...,es,0.002744
63805,63805,"Amigo mio, quien diría que esta travesía de lo...",es,0.003116


In [4]:
# translated_toxic = pd.read_csv('../data/translated_2018/{}_distill.csv'.format(lang)) #.sample(frac=0.43)
# translated_toxic['lang'] = lang
translated_toxic = pd.read_csv('../data/translated_2018/combined_distilled.csv') 
translated_toxic = translated_toxic[translated_toxic.lang.isin(lang_list)].sample(frac=0.43)
translated_toxic = translated_toxic[['id', 'comment_text', 'lang', 'toxic']]
translated_toxic

,id,comment_text,lang,toxic
121208,2043f6becc9a013b,Traduciendo Benjamin Moore Norman\n\nHola. Veo...,es,0.000034
192140,32f694f9dbaa4204,"""\n\n No destroce las páginas, como hizo con e...",es,0.000256
886966,ed85eb6d2aa372a8,"""== Esta página no debe eliminarse rápidamente...",es,0.000032
411602,6dcfbf206e8f65d7,Algunas de las fuentes están vinculadas direct...,es,0.000127
488042,82872aec66aa7c90,"""\neditor """" wipe ass """"\n\nyippee yehaw okie ...",es,0.783864
...,...,...,...,...
745232,c792e13c25bbd3ac,Quizás controversias sobre la vacuna. (hablar ...,es,0.000068
178406,2f4faf99f6a31379,"¿PUEDE, POR FAVOR, EXPLICARME DÓNDE ESTÁ EL MA...",es,0.989707
634196,a9c1d3532869d6c6,"En los últimos 9 meses, xwomanizerx ha realiza...",es,0.349370
867248,e8407e85dc44fb61,No estaba acosando al usuario: Active Banana. ...,es,0.002864


In [5]:
pd.concat([language_df, translated_toxic]).reset_index(drop=True).to_csv('../data/{}_all.csv'.format(lang), index=False)
# translated_toxic.to_csv('../data/{}_all.csv'.format(lang), index=False)

In [6]:
preds_df = pd.read_csv('../data/{}_preds.csv'.format(lang))
preds_df

,id,toxic
0,7,0.023841
1,8,0.452782
2,9,0.122170
3,22,0.013692
4,49,0.103154
...,...,...
8433,63783,0.108608
8434,63787,0.478479
8435,63804,0.036656
8436,63805,0.162108


In [7]:
preds_df['raw_toxic'] = language_df.reset_index()['toxic']
preds_df

,id,toxic,raw_toxic
0,7,0.023841,0.046024
1,8,0.452782,0.269433
2,9,0.122170,0.281472
3,22,0.013692,0.003624
4,49,0.103154,0.154248
...,...,...,...
8433,63783,0.108608,0.046097
8434,63787,0.478479,0.380755
8435,63804,0.036656,0.002744
8436,63805,0.162108,0.003116


In [13]:
preds_df[(preds_df.toxic - preds_df.raw_toxic).abs() > 0.3] #.sample(n=1)

,id,toxic,raw_toxic,combined
18,159,0.880596,0.571281,0.633144
34,321,0.654668,0.105756,0.215538
57,537,0.698151,0.335233,0.407816
70,586,0.676147,0.226599,0.316509
94,768,0.397382,0.027364,0.101368
...,...,...,...,...
8244,62368,0.570942,0.212542,0.284222
8260,62513,0.033280,0.335695,0.275212
8266,62536,0.720805,0.411756,0.473566
8340,63111,0.101049,0.521372,0.437308


In [ ]:
test_df[test_df.id == 9].content.values[0]

In [10]:
preds_df['combined'] = 0.2 * preds_df.toxic + 0.8 * preds_df.raw_toxic

In [ ]:
preds_df

In [11]:
test_df.loc[preds_df.id.values, 'toxic'] = preds_df['combined'].values
# test_df.loc[preds_df.id.values, 'toxic'] = preds_df['toxic'].values

In [12]:
test_df.loc[:, ['id', 'toxic']].to_csv('../data/submit9537_{}.csv'.format(lang), index=False)